# LSTM Training

In [1]:
# No need to do this in Azure
# Replace this with the actual directory containing the notebook and Python scripts
# %cd stock-pred

In [2]:
from azureml.core import Dataset, Workspace

ws = Workspace.from_config()
ds = Dataset.get_by_name(ws, name='stock_prices')
# ds_mount = ds.as_mount(path_on_compute='/dataset/')
ds.download('./dataset/', overwrite=True)
! ls -la ./dataset

# Global settings
WINDOW = 50
EPOCHS = 15
BATCH = 20
TEST_RATIO = .2

DATA_PATH = './dataset/20220512_tesla.pkl'
TRAINED_ROOT = './trained/'
SCALER_PATH = TRAINED_ROOT +  '/scaler.pkl'
MODEL_PATH = TRAINED_ROOT + '/model'
MODEL_NAME = 'stock-pred-lstm'
TENSORBOARD_PATH = './tensorboard'

# Forces to use CPU rather than GPU
# NVIDIA drivers of higher versions have messy implimentation of LSTM!
# Ref: https://github.com/mozilla/DeepSpeech/issues/3088#issuecomment-656056969
# Ref: https://github.com/tensorflow/tensorflow/issues/35950#issuecomment-577427083
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

total 19
drwxrwxrwx 2 root root     0 Jun 13 10:17 .
drwxrwxrwx 2 root root     0 Jun 13 06:47 ..
-rwxrwxrwx 1 root root   315 Jun 13 10:22 .amlignore
-rwxrwxrwx 1 root root   315 Jun 13 10:22 .amlignore.amltmp
-rwxrwxrwx 1 root root 17926 Jun 14 01:13 20220512_tesla.pkl


## Load and Preprocess Data

In [3]:
from train import (load_data, transform_data, extract_x_y)

stock_dates, stock_prices = load_data(DATA_PATH)
scaler, scaled_data = transform_data(stock_prices)

train_split = int(len(scaled_data) * (1.0 - TEST_RATIO))
scaled_data_train = scaled_data[:train_split]
x_train, y_train = extract_x_y(scaled_data_train, window=WINDOW, offset=WINDOW)

## Train Model

In [4]:
from train import (build_LSTM, train_model)

model = build_LSTM(x_train, units=WINDOW)
train_model(model, x_train, y_train, EPOCHS, BATCH, interactive_progress=True, tensorboard_path=TENSORBOARD_PATH)

Epoch 1/15
7/7 [==============================] - 4s 198ms/step - loss: 0.8395 - val_loss: 0.0918
Epoch 2/15
7/7 [==============================] - 0s 35ms/step - loss: 0.3054 - val_loss: 0.5846
Epoch 3/15
7/7 [==============================] - 0s 35ms/step - loss: 0.2674 - val_loss: 0.2533
Epoch 4/15
7/7 [==============================] - 0s 36ms/step - loss: 0.2012 - val_loss: 0.1785
Epoch 5/15
7/7 [==============================] - 0s 36ms/step - loss: 0.1771 - val_loss: 0.2640
Epoch 6/15
7/7 [==============================] - 0s 35ms/step - loss: 0.1551 - val_loss: 0.2996
Epoch 7/15
7/7 [==============================] - 0s 36ms/step - loss: 0.1412 - val_loss: 0.2862
Epoch 8/15
7/7 [==============================] - 0s 36ms/st

## Test Model

In [5]:
from train import (predict, get_rmse, get_mape)

x_test, _ = extract_x_y(scaled_data, window=WINDOW, offset=train_split)
y_lstm_scaled = model.predict(x_test)
y_lstm = scaler.inverse_transform(y_lstm_scaled)
y_actual = stock_prices[train_split:]

rmse_lstm = get_rmse(pred=y_lstm, actual=y_actual)
mape_lstm = get_mape(pred=y_lstm, actual=y_actual)
print(f'RMSE: {rmse_lstm}, MAPE: {mape_lstm}')

RMSE: 131.68806420557846, MAPE: 0.1210426860978854


In [6]:
import matplotlib.pyplot as plt

def plot_stock(dates, y_train, y_pred, y_actual, train_pos):
    fig = plt.figure(figsize=(20, 10))
    plt.plot(dates[:train_pos], y_train, label='Train Closing Price')
    plt.plot(dates[train_pos:], y_actual, label='Test Closing Price')
    plt.plot(dates[train_pos:], y_pred, label='Pred Closing Price')
    plt.title('Stock Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend(loc='upper left')

plot_stock(stock_dates, stock_prices[:train_split], y_lstm, y_actual, train_split)

## Save Scaler and Model

In [7]:
from train import (save_scaler, save_model)

save_scaler(scaler, SCALER_PATH)
save_model(model, MODEL_PATH)

INFO:tensorflow:Assets written to: ./trained//model/assets


INFO:tensorflow:Assets written to: ./trained//model/assets


In [8]:
from azureml.core.model import Model

model = Model.register(workspace=ws,
                       model_path=TRAINED_ROOT,
                       model_name=MODEL_NAME,
                       model_framework=Model.Framework.TENSORFLOW)

Registering model stock-pred-lstm
